# FAQ Update Pipeline - Content Change Application

**Apply detected content changes to FAQ database**

## Overview

This notebook implements the **FAQ update workflow** by reading from `content_change_log` and applying changes:

1. **new_content** → Generate new Q&A pairs and link to new checksum
2. **deleted_content** → Inactivate sole-source FAQs OR regenerate multi-source FAQs
3. **modified_content** → Evaluate and regenerate affected FAQs, roll provenance forward

### Architecture

```
content_change_log (input)
    ↓
Change Type Router
    ↓
├─ NEW_CONTENT → generate_faq_for_new_content()
├─ DELETED_CONTENT → handle_deleted_content()
└─ MODIFIED_CONTENT → handle_modified_content()
    ↓
FAQ Tables (updated)
```

---

## Step 1: Setup and Configuration

In [ ]:
# Configuration
import sys
import os
from pathlib import Path
from datetime import datetime
import uuid

# Add project root to path
NOTEBOOK_DIR = Path(os.getcwd())
PROJECT_ROOT = NOTEBOOK_DIR if NOTEBOOK_DIR.name == "faq_update" else NOTEBOOK_DIR
sys.path.insert(0, str(PROJECT_ROOT))

# Import environment utilities
from utility.environment import is_databricks

# Detect environment
IS_DATABRICKS = is_databricks()

if IS_DATABRICKS:
    # Databricks configuration
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    dbutils.widgets.text("volume", "datacloud-dita-gen-volume", "Volume")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    VOLUME = dbutils.widgets.get("volume")
    
    PROJECT_ROOT = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update"
    
    print(f"🏢 Databricks mode: {CATALOG}.{SCHEMA}")
    print(f"📁 Project root: {PROJECT_ROOT}")
else:
    # Local configuration
    CATALOG = None
    SCHEMA = None
    VOLUME = None
    
    DATABASE_DIR = PROJECT_ROOT / "databases"
    DB_PATH = str(DATABASE_DIR / "faq_update.db")
    
    print(f"💻 Local mode")
    print(f"📁 Database: {DB_PATH}")
    print(f"📁 Project root: {PROJECT_ROOT}")

## Step 2: Initialize Backend and Repositories

In [ ]:
from database.backends.factory import BackendFactory
from database.config import DatabaseConfig
from database.repository import AuditRepository, ContentRepository

print("="*80)
print("🔧 INITIALIZING DATABASE BACKEND AND REPOSITORIES")
print("="*80)

# Create configuration based on environment
if IS_DATABRICKS:
    config = DatabaseConfig(
        backend='databricks',
        catalog=CATALOG,
        schema=SCHEMA
    )
    print(f"✓ Configuration: Databricks ({CATALOG}.{SCHEMA})")
else:
    config = DatabaseConfig(
        backend='sqlite',
        db_path=DB_PATH
    )
    print(f"✓ Configuration: SQLite ({DB_PATH})")

# Create backend
backend = BackendFactory.create_backend(config)
print(f"✓ Backend created: {backend.__class__.__name__}")

# Create repositories
audit_repo = AuditRepository(backend)
content_repo = ContentRepository(backend)
print(f"✓ Repositories initialized")

print("="*80)
print("✅ Database backend and repositories ready!")
print("="*80)

## Step 3: Load Pending Changes from content_change_log

Load all unprocessed changes that need to be applied to FAQ database.

In [ ]:
import pandas as pd

print("\n📊 Loading unprocessed changes from content_change_log...")

# Query for unprocessed changes using the processed flag
# A change is considered "unprocessed" if processed=0
query = """
    SELECT
        c.change_id,
        c.file_name,
        c.change_type,
        c.content_checksum,
        c.previous_checksum,
        c.similarity_score,
        c.diff_data,
        c.detection_run_id,
        c.detection_timestamp
    FROM content_change_log c
    WHERE c.processed = 0
      AND c.requires_faq_regeneration = 1
    ORDER BY c.change_id
"""

changes = backend.execute_query(query)
changes_df = pd.DataFrame(changes)

print(f"Found {len(changes_df)} unprocessed changes (processed=0)")

if len(changes_df) > 0:
    # Show summary by change type
    print("\nChanges by type:")
    print(changes_df['change_type'].value_counts())

    print("\nSample changes:")
    display(changes_df.head(10))
else:
    print("\n⊘ No unprocessed changes found.")
    print("All changes in content_change_log have been processed.")

    # Show summary of processed changes
    processed_query = """
        SELECT
            COUNT(*) as total_processed,
            SUM(CASE WHEN processing_error IS NOT NULL THEN 1 ELSE 0 END) as total_failed
        FROM content_change_log
        WHERE processed = 1
    """
    processed = backend.execute_query(processed_query)
    if processed and len(processed) > 0:
        print(f"\nTotal processed changes: {processed[0]['total_processed']}")
        print(f"Total failed changes: {processed[0]['total_failed']}")

## Step 4: Define FAQ Generation Functions

Helper functions to generate Q&A pairs (adapted from notebooks 3 & 4).

In [ ]:
# ============================================================================
# FAQ Generation Functions - Using faq_generation Module
# ============================================================================

from faq_generation import QuestionGenerator, AnswerGenerator

# Initialize generators (singleton pattern - create once, use multiple times)
_question_generator = None
_answer_generator = None

def get_question_generator():
    """Get or create the global question generator instance (lazy initialization)."""
    global _question_generator
    if _question_generator is None:
        _question_generator = QuestionGenerator(
            chat_model_name="gpt-4o_2024-11-20-pgo-amrs",
            temperature=0.0,
            max_tokens=4000,
            max_retries=10,
            seconds_between_retries=5,
            max_questions=5
        )
    return _question_generator


def get_answer_generator():
    """Get or create the global answer generator instance (lazy initialization)."""
    global _answer_generator
    if _answer_generator is None:
        _answer_generator = AnswerGenerator(
            chat_model_name="gpt-4o_2024-11-20-pgo-amrs",
            temperature=0.0,
            max_tokens=4000,
            max_retries=10,
            seconds_between_retries=5,
            use_embeddings=False  # Don't generate embeddings for single question processing
        )
    return _answer_generator


def generate_questions_for_chunk(chunk_text: str, content_checksum: str) -> list:
    """
    Generate questions for a content chunk using LLM.
    
    This function uses the faq_generation module for question generation,
    which is the same code used in notebook 3_question_source_gen.ipynb.
    
    Args:
        chunk_text: Text content to generate questions from
        content_checksum: Checksum of the content chunk
    
    Returns:
        List of dicts with question data: [{'question_text': '...', 'question_id': ...}, ...]
    """
    print(f"  Generating questions for checksum {content_checksum[:8]}...")
    
    try:
        # Get the question generator
        gen = get_question_generator()
        
        # Generate questions using the faq_generation module
        questions = gen.generate_from_text(
            chunk_text=chunk_text,
            content_checksum=content_checksum,
            source_type='document',
            generation_method='LLM',
            status='active'
        )
        
        print(f"  ✓ Generated {len(questions)} questions")
        return questions
        
    except Exception as e:
        print(f"  ❌ Error generating questions: {e}")
        # Return empty list on error (graceful degradation)
        return []


def generate_answers_for_question(question_text: str, chunk_text: str, question_id: str = None, content_checksum: str = None) -> list:
    """
    Generate answers for a question using LLM.
    
    This function uses the faq_generation module for answer generation,
    which is the same code used in notebook 4_answer_source_gen.ipynb.
    
    Args:
        question_text: Question to answer
        chunk_text: Source content for answer generation
        question_id: ID of the question (optional)
        content_checksum: Checksum of the content (optional)
    
    Returns:
        List of dicts with answer data: [{'answer_text': '...', 'confidence_score': ...}, ...]
    """
    print(f"  Generating answer for question: {question_text[:50]}...")
    
    try:
        # Get the answer generator
        gen = get_answer_generator()
        
        # Generate a unique ID if not provided
        if question_id is None:
            import uuid
            question_id = str(uuid.uuid4())
        
        if content_checksum is None:
            from utility import compute_checksum
            content_checksum = compute_checksum(chunk_text)
        
        # Generate answer using the faq_generation module
        answer = gen.generate_from_question(
            question_text=question_text,
            context_text=chunk_text,
            question_id=question_id,
            content_checksum=content_checksum,
            status='active',
            confidence_score=0.95
        )
        
        # Wrap in list to maintain backward compatibility
        answers = [answer]
        
        print(f"  ✓ Generated answer")
        return answers
        
    except Exception as e:
        print(f"  ❌ Error generating answer: {e}")
        # Return empty list on error (graceful degradation)
        return []

print("✓ FAQ generation functions defined using faq_generation module")

## Step 5: Define Change Handlers

Implement the three main handlers for each change type.

In [ ]:
# ============================================================================
# Import Simplified Content Handlers with Best Practices
# ============================================================================

from faq_impact.analysis.strategies.simple_new_content_handler import SimpleNewContentHandler
from faq_impact.analysis.strategies.simple_deleted_content_handler import SimpleDeletedContentHandler
from faq_impact.analysis.strategies.simple_modified_content_handler import SimpleModifiedContentHandler

# Initialize handlers (create once, reuse for all changes)
print("Initializing content handlers...")

new_content_handler = SimpleNewContentHandler(
    backend=backend,
    question_generator=get_question_generator(),
    answer_generator=get_answer_generator(),
    use_transactions=True  # Enable transactions for data integrity
)

deleted_content_handler = SimpleDeletedContentHandler(
    backend=backend,
    question_generator=get_question_generator(),
    answer_generator=get_answer_generator(),
    use_transactions=True  # Enable transactions for data integrity
)

modified_content_handler = SimpleModifiedContentHandler(
    backend=backend,
    question_generator=get_question_generator(),
    answer_generator=get_answer_generator(),
    use_transactions=True,  # Enable transactions for data integrity
    similarity_threshold=0.6  # Major change threshold
)

print("✓ SimpleNewContentHandler initialized")
print("✓ SimpleDeletedContentHandler initialized")
print("✓ SimpleModifiedContentHandler initialized")


def handle_new_content(change: dict) -> dict:
    """
    Handle NEW_CONTENT change using SimpleNewContentHandler.
    
    This is now a thin wrapper around SimpleNewContentHandler which provides:
    - ✅ Duplicate detection and prevention
    - ✅ Comprehensive validation
    - ✅ Cost estimation
    - ✅ Detailed logging
    - ✅ Idempotency (safe to re-run)
    - ✅ Graceful error handling
    
    Steps performed by handler:
    1. Validate checksum format
    2. Check for duplicates (skip if already processed)
    3. Estimate generation cost
    4. Get chunk text from content_chunks
    5. Generate questions from chunk
    6. Generate answers for each question
    7. Persist Q/A and link sources to checksum
    
    Args:
        change: Change dictionary with keys:
            - change_id: int
            - content_checksum: str (64-char SHA-256 hex)
            - file_name: str (optional)
    
    Returns:
        Dict with:
            - success: bool
            - questions_created: int
            - answers_created: int
            - estimated_cost_usd: float
            - skipped: bool
            - skip_reason: str (if skipped)
            - error: str (if failed)
    """
    print(f"\n[NEW_CONTENT] Processing change_id={change['change_id']}")
    
    # Use handler to process change
    result = new_content_handler.handle(change)
    
    # Convert HandlerResult to dict for backward compatibility
    result_dict = result.to_dict()
    
    # Log result
    if result.success:
        if result.skipped:
            print(f"  ⊘ Skipped: {result.skip_reason}")
        else:
            print(f"  ✓ Created {result.questions_created} questions, {result.answers_created} answers")
            print(f"  💰 Estimated cost: ${result.estimated_cost_usd:.2f}")
    else:
        print(f"  ✗ Error: {result.error}")
    
    return result_dict


def handle_deleted_content(change: dict) -> dict:
    """
    Handle DELETED_CONTENT change using SimpleDeletedContentHandler.
    
    This is now a thin wrapper around SimpleDeletedContentHandler which provides:
    - ✅ Source counting (sole-source vs multi-source detection)
    - ✅ Cascading inactivations (question → answers)
    - ✅ Answer regeneration for multi-source scenarios
    - ✅ Orphan detection (0 valid sources)
    - ✅ Comprehensive validation
    - ✅ Detailed logging
    - ✅ Graceful error handling
    
    Steps performed by handler:
    1. Find questions linked to deleted checksum
    2. For each question:
        - Count remaining valid sources
        - If sole source (1) or orphaned (0) → inactivate question + cascade to answers
        - If multi-source (2+) → regenerate question + regenerate answers
    3. Invalidate deleted source links
    4. Return detailed metrics
    
    Decision Logic:
        - 0 sources (orphaned) → INACTIVATE question + cascade to all answers
        - 1 source (sole source) → INACTIVATE question + cascade to all answers
        - 2+ sources (multi-source) → REGEN question + REGEN affected answers
    
    Args:
        change: Change dictionary with keys:
            - change_id: int
            - previous_checksum: str (64-char SHA-256 hex of deleted content)
            - file_name: str (optional)
    
    Returns:
        Dict with:
            - success: bool
            - questions_inactivated: int
            - questions_regenerated: int
            - answers_inactivated: int
            - answers_regenerated: int
            - no_impact: bool (if no FAQs affected)
            - error: str (if failed)
    """
    print(f"\n[DELETED_CONTENT] Processing change_id={change['change_id']}")
    
    # Use handler to process change
    result = deleted_content_handler.handle(change)
    
    # Convert HandlerResult to dict for backward compatibility
    result_dict = result.to_dict()
    
    # Log result
    if result.success:
        if result.no_impact:
            print(f"  ⊘ No FAQs affected by deleted content")
        else:
            print(f"  ✓ Inactivated {result.questions_inactivated} questions, {result.answers_inactivated} answers")
            print(f"  ✓ Regenerated {result.questions_regenerated} questions, {result.answers_regenerated} answers")
    else:
        print(f"  ✗ Error: {result.error}")
    
    return result_dict


def handle_modified_content(change: dict) -> dict:
    """
    Handle MODIFIED_CONTENT change using SimpleModifiedContentHandler.
    
    This is now a thin wrapper around SimpleModifiedContentHandler which provides:
    - ✅ Similarity-based decisions (major vs minor changes using 0.6 threshold)
    - ✅ Source counting (sole-source vs multi-source detection)
    - ✅ Provenance rolling (updates source links old → new checksum)
    - ✅ Complete answer regeneration (fully implemented, not TODO)
    - ✅ Comprehensive validation
    - ✅ Detailed logging
    - ✅ Graceful error handling
    
    Steps performed by handler:
    1. Find questions linked to old content (previous_checksum)
    2. For each question:
        - Count valid sources
        - If sole source (1) → regenerate question
        - If multi-source (2+) → skip question regeneration
        - Regenerate all affected answers (based on similarity score)
        - Roll provenance forward (old → new checksum)
    3. Return detailed metrics
    
    Decision Logic:
        - Sole source question → REGEN question
        - Multi-source question → skip question, check answers
        - similarity < 0.6 (major change) → REGEN answers (high priority)
        - similarity ≥ 0.6 (minor change) → REGEN answers (standard priority)
        - All questions → roll provenance forward (old → new checksum)
    
    Args:
        change: Change dictionary with keys:
            - change_id: int
            - previous_checksum: str (64-char SHA-256 hex of old content)
            - content_checksum: str (64-char SHA-256 hex of new content)
            - similarity_score: float (0.0-1.0)
            - file_name: str (optional)
    
    Returns:
        Dict with:
            - success: bool
            - questions_regenerated: int
            - answers_regenerated: int
            - provenance_rolled: int
            - major_changes: int (number of major changes)
            - minor_changes: int (number of minor changes)
            - no_impact: bool (if no FAQs affected)
            - error: str (if failed)
    """
    print(f"\n[MODIFIED_CONTENT] Processing change_id={change['change_id']}")
    
    # Use handler to process change
    result = modified_content_handler.handle(change)
    
    # Convert HandlerResult to dict for backward compatibility
    result_dict = result.to_dict()
    
    # Log result
    if result.success:
        if result.no_impact:
            print(f"  ⊘ No FAQs affected by modified content")
        else:
            print(f"  ✓ Regenerated {result.questions_regenerated} questions, {result.answers_regenerated} answers")
            print(f"  ✓ Rolled provenance for {result.provenance_rolled} questions")
            if result.major_changes > 0 or result.minor_changes > 0:
                print(f"  ℹ️  Major changes: {result.major_changes}, Minor changes: {result.minor_changes}")
    else:
        print(f"  ✗ Error: {result.error}")
    
    return result_dict

print("✓ All change handlers defined with simplified handlers")

## Step 6A: Initialize Decision Repository

Create the decision repository to track what will be done before execution.

In [ ]:
# ============================================================================
# Initialize Decision Repository
# ============================================================================

from faq_impact.database.repository.decision_repository import DecisionRepository

# Create decision repository
decision_repo = DecisionRepository(backend)

print("✓ DecisionRepository initialized")

# Reinitialize handlers WITH decision_repo for decision tracking
print("\nReinitializing handlers with decision repository...")

new_content_handler = SimpleNewContentHandler(
    backend=backend,
    question_generator=get_question_generator(),
    answer_generator=get_answer_generator(),
    decision_repo=decision_repo,  # NEW: Enable decision tracking
    use_transactions=True
)

deleted_content_handler = SimpleDeletedContentHandler(
    backend=backend,
    question_generator=get_question_generator(),
    answer_generator=get_answer_generator(),
    decision_repo=decision_repo,  # NEW: Enable decision tracking
    use_transactions=True
)

modified_content_handler = SimpleModifiedContentHandler(
    backend=backend,
    question_generator=get_question_generator(),
    answer_generator=get_answer_generator(),
    decision_repo=decision_repo,  # NEW: Enable decision tracking
    use_transactions=True,
    similarity_threshold=0.6
)

print("✓ Handlers reinitialized with decision tracking enabled")
print("\n" + "="*80)
print("✅ DECISION REPOSITORY READY")
print("="*80)

## Step 6B: PHASE 1 - Analyze All Changes (No Execution)

**NEW WORKFLOW**: Separate analysis from execution for full visibility

This phase analyzes ALL changes and creates decisions WITHOUT modifying the database.
Benefits:
- 🔍 Preview impact before execution
- 💰 See estimated costs
- 📋 Track WHY each decision was made
- ✅ User approval before execution

## Step 6C: PHASE 2 - Persist Decisions to Database

Save all decisions to `faq_impact_decisions` table for audit trail.

In [ ]:
print("="*80)
print("PHASE 2: SAVING DECISIONS TO DATABASE")
print("="*80)

if len(all_decisions) == 0:
    print("⊘ No decisions to save")
else:
    try:
        decision_ids = decision_repo.insert_batch(all_decisions)
        print(f"✓ Saved {len(decision_ids)} decisions to faq_impact_decisions table")
        print("\n" + "="*80)
        print("✅ PHASE 2 COMPLETE - DECISIONS PERSISTED")
        print("="*80)
    except Exception as e:
        print(f"❌ Error saving decisions: {e}")
        print("\n⚠️ Continuing anyway - decisions are in memory")

In [ ]:
print("="*80)
print("PHASE 3: IMPACT PREVIEW")
print("="*80)

if len(all_decisions) == 0:
    print("⊘ No decisions to execute")
else:
    # Count by action and entity type
    from collections import Counter
    
    action_counts = Counter(d['action'] for d in all_decisions)
    entity_counts = Counter(d['entity_type'] for d in all_decisions)
    reason_counts = Counter(d['reason_code'] for d in all_decisions)
    
    print("\n📊 DECISIONS BY ACTION:")
    for action, count in action_counts.most_common():
        print(f"  {action}: {count}")
    
    print("\n📊 DECISIONS BY ENTITY TYPE:")
    for entity_type, count in entity_counts.most_common():
        print(f"  {entity_type}: {count}")
    
    print("\n📊 DECISIONS BY REASON:")
    for reason, count in reason_counts.most_common():
        print(f"  {reason}: {count}")
    
    # Calculate total estimated cost
    total_cost = sum(d.get('estimated_cost_usd', 0.0) for d in all_decisions)
    
    print("\n💰 COST ESTIMATE:")
    print(f"  Total estimated cost: ${total_cost:.2f}")
    
    # Break down by change type
    print("\n📋 BREAKDOWN BY CHANGE TYPE:")
    change_type_groups = {}
    for d in all_decisions:
        ct = d.get('change_type', 'unknown')
        if ct not in change_type_groups:
            change_type_groups[ct] = []
        change_type_groups[ct].append(d)
    
    for change_type, decisions in change_type_groups.items():
        cost = sum(d.get('estimated_cost_usd', 0.0) for d in decisions)
        print(f"  {change_type}: {len(decisions)} decisions, ${cost:.2f}")
    
    # Show sample decisions
    print("\n📝 SAMPLE DECISIONS (first 10):")
    for i, d in enumerate(all_decisions[:10]):
        print(f"  [{i+1}] {d['entity_type']} → {d['action']} (reason: {d['reason_code']})")
    
    if len(all_decisions) > 10:
        print(f"  ... and {len(all_decisions) - 10} more")

print("\n" + "="*80)
print("✅ PHASE 3 COMPLETE - IMPACT PREVIEW SHOWN")
print("="*80)

## Step 6E: PHASE 4 - Execute Decisions (WITH USER APPROVAL)

**IMPORTANT**: This phase actually modifies the database. Only run after reviewing the preview above.

## Step 7: View Results Summary

In [ ]:
# This cell shows execution results from PHASE 4
# Results are displayed at the end of Phase 4 execution

print("="*80)
print("PROCESSING UNPROCESSED CHANGES")
print("="*80)

# Process each change directly (no pre-analysis phase)
execution_results = []

for idx, change in changes_df.iterrows():
    change_dict = change.to_dict()
    change_type = change['change_type']
    change_id = change['change_id']
    
    try:
        print(f"\n[{idx+1}/{len(changes_df)}] Processing {change_type} (change_id={change_id})...")
        
        # Call handler (which now does: execute + log POST-EXEC decisions + mark processed)
        if change_type == 'new_content':
            result = new_content_handler.handle(change_dict)
        elif change_type == 'deleted_content':
            result = deleted_content_handler.handle(change_dict)
        elif change_type == 'modified_content':
            result = modified_content_handler.handle(change_dict)
        else:
            print(f"  ⚠️ Unknown change_type: {change_type}")
            continue
        
        result_dict = result.to_dict()
        result_dict['change_id'] = change_id
        result_dict['change_type'] = change_type
        execution_results.append(result_dict)
        
        if result.success:
            print(f"  ✓ Success")
        else:
            print(f"  ✗ Failed: {result.error}")
    
    except Exception as e:
        print(f"  ❌ Error: {e}")
        execution_results.append({
            'change_id': change_id,
            'change_type': change_type,
            'success': False,
            'error': str(e)
        })

print("\n" + "="*80)
print("✅ PROCESSING COMPLETE")
print("="*80)

# Show results
if len(execution_results) > 0:
    results_df = pd.DataFrame(execution_results)
    display(results_df)